In [1]:
import pandas as pd
import numpy as np


import sqlalchemy
from sqlalchemy import create_engine, inspect, Column, Integer, Float, String, Sequence, Boolean, text
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base

In [2]:
#Using declarative base
Base = declarative_base()
engine = create_engine('sqlite:///lyrics3.db', echo = False)

In [3]:
#Creating the table metadata
Base.metadata.create_all(engine)
session = Session(engine)

In [4]:
#Binding engine to the metadata
meta = sqlalchemy.MetaData()
meta.reflect(bind = engine)

In [5]:
lyrics = pd.read_sql_table('lyrics', engine, index_col="id", coerce_float=False)
tracks = pd.read_sql_table('tracks', engine, index_col="id", coerce_float=False)
genre = pd.read_sql_table('genres', engine, index_col="id", coerce_float=False)

In [6]:
lyrics = lyrics[lyrics['lyrics_language']=='en']

In [7]:
genre['album_id']=genre['album_id'].astype(int)
genre.head()

,album_id,album_name,artist,genre
id,,,,
1,10266031,Big Willie Style,Will Smith,Rap
2,10266041,War,U2,Pop/Rock
3,10266127,Led Zeppelin,Led Zeppelin,Pop/Rock
4,10266180,Power Windows,Rush,Pop/Rock
5,10266202,Willennium,Will Smith,Rap


In [8]:
lyrics_tracks = pd.merge(tracks, lyrics, on="lyrics_id")
lyrics_tracks.head()

,album_id,artist_id,artist_name,explicit_x,has_lyrics,lyrics_id,track_id,track_name,explicit_y,lyrics_body,lyrics_language,lyrics_language_description
0,29219288,33491453,Drake,1,1,18083442,152383631,In My Feelings,1,"Trap, TrapMoneyBenny\nThis shit got me in my f...",en,English
1,29504205,36425860,"DJ Khaled feat. Justin Bieber, Quavo & Chance ...",1,1,18105233,154318893,No Brainer,1,We the Best Music!\nAnother one!\nDJ Khaled!\n...,en,English
2,29005911,36070710,Maroon 5 feat. Cardi B,1,1,18066274,150959640,Girls Like You,1,"Spent twenty-four hours, I need more hours wit...",en,English
3,29211432,36070710,Maroon 5 feat. Cardi B,1,1,18066274,152271189,Girls Like You,1,"Spent twenty-four hours, I need more hours wit...",en,English
4,28578797,35716462,Cardi B feat. Bad Bunny & J Balvin,1,1,17942796,148363403,I Like It,1,"Yeah baby, I like it like that\nYou gotta beli...",en,English


In [9]:
data = pd.merge(lyrics_tracks, genre, on="album_id")

In [27]:
data = data.sample(2000)

In [28]:
lyrics_genre = data[['artist_name','album_name','track_name','lyrics_body','genre']]

In [29]:
lyrics_genre['combined_text']= lyrics_genre['artist_name']+ " " + lyrics_genre['album_name']+" " + lyrics_genre['track_name']+" " + lyrics_genre['lyrics_body']

C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
lyrics_genre.head()

,artist_name,album_name,track_name,lyrics_body,genre,combined_text
7481,"Bonnie ""Prince"" Billy",Lie Down In the Light,You Want That Picture,What did you do when you saw that I'd gone\ndi...,Pop/Rock,"Bonnie ""Prince"" Billy Lie Down In the Light Yo..."
6262,Prince,Work It Volume 2,Heaven,,Pop/Rock,Prince Work It Volume 2 Heaven
8743,Beyoncé feat. Drake,Beyoncé,Mine,I've been watching for the signs\nTook a trip ...,R&B,Beyoncé feat. Drake Beyoncé Mine I've been wat...
1719,Leo Sayer,Don't Wait Until Tomorrow,Long Tall Glasses,I was travellin' down the road\nFeelin' hungry...,Pop/Rock,Leo Sayer Don't Wait Until Tomorrow Long Tall ...
1419,Slayer,Christ Illusion,Jihad,I have witnessed your death\nI've seen it many...,Pop/Rock,Slayer Christ Illusion Jihad I have witnessed ...


In [31]:
lyrics_genre.drop(labels=['artist_name','album_name','track_name','lyrics_body'], axis=1, inplace=True)

C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
lyrics_genre.head()

,genre,combined_text
7481,Pop/Rock,"Bonnie ""Prince"" Billy Lie Down In the Light Yo..."
6262,Pop/Rock,Prince Work It Volume 2 Heaven
8743,R&B,Beyoncé feat. Drake Beyoncé Mine I've been wat...
1719,Pop/Rock,Leo Sayer Don't Wait Until Tomorrow Long Tall ...
1419,Pop/Rock,Slayer Christ Illusion Jihad I have witnessed ...


In [33]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [34]:
lyrics_genre.dropna(inplace=True)
lyrics_genre['is_english'] = lyrics_genre['combined_text'].apply(lambda x: isEnglish(x))
lyrics_genre.head()

C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,genre,combined_text,is_english
7481,Pop/Rock,"Bonnie ""Prince"" Billy Lie Down In the Light Yo...",True
6262,Pop/Rock,Prince Work It Volume 2 Heaven,True
8743,R&B,Beyoncé feat. Drake Beyoncé Mine I've been wat...,False
1719,Pop/Rock,Leo Sayer Don't Wait Until Tomorrow Long Tall ...,True
1419,Pop/Rock,Slayer Christ Illusion Jihad I have witnessed ...,True


In [35]:
lyrics_genre = lyrics_genre[lyrics_genre['is_english']==True]

In [36]:
lyrics_genre['is_english'].value_counts()

True    1793
Name: is_english, dtype: int64

In [37]:
#dependencies
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, PorterStemmer
import nltk
import string
import re
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
# defining the stopwords
stopword = stopwords.words('english')

#instantiating lemmatization and stemming objects
wn = WordNetLemmatizer()
ps = PorterStemmer()

In [39]:
def clean_text(text):
    '''
    Function accepts a text input and does three things:
    1. Removes punctuation
    2. Splits into tokens
    3. Removes tokens that are stopwords, conducts stemming, and joins together into a single string
    '''
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopword])
    return text

In [40]:
#cleans text using clean_text() function
lyrics_genre['body_text_clean'] = lyrics_genre['combined_text'].apply(lambda x: clean_text(x))

In [41]:
ngram_vect = CountVectorizer(ngram_range=(1,2))

In [42]:
X_counts = ngram_vect.fit_transform(lyrics_genre['body_text_clean'])

In [43]:
print(X_counts.shape)

(1793, 65379)


In [44]:
X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = ngram_vect.get_feature_names()

In [45]:
X_counts_df.head()

,000,000 feet,000 kiss,007,007 im,04,04 v6,0f,0f lyric,10,...,zooey,zooey deschanel,zoom,zoom impend,zooropa,zooropa dirti,zooropa lemon,zooropa numb,zs,zs rock
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
y = lyrics_genre[['genre']]

y_genres = pd.get_dummies(y)
y_genres = y_genres.values.reshape(-1,1)

In [28]:
y_genres.to_csv('y_vectorized_genres.csv', encoding='utf-8')
X_counts_df.to_csv('X_counts_genres.csv', encoding='utf-8')

KeyboardInterrupt: 

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_counts, y_genres, stratify=y)

ValueError: Found input variables with inconsistent numbers of samples: [1793, 26895]

In [48]:
from sklearn.linear_model import LogisticRegression

log_classifier = LogisticRegression()

In [50]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train.toarray())
y_scaler = StandardScaler().fit(y_train)

C:\Users\micha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [54]:
X_train_scaled = X_scaler.transform(X_train.toarray())
X_test_scaled = X_scaler.transform(X_test.toarray())
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

C:\Users\micha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [55]:
log_classifier.fit(X_train, y_train)

ValueError: bad input shape (1344, 15)